This notebook builds on EDA done here: https://github.com/fractaldatalearning/Capstone2/blob/main/notebooks/eda3_1user_modeling.ipynb

In [1]:
import pandas as pd
import numpy as np
import os
import random

import matplotlib.pyplot as plt
import seaborn as sns

import collections

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
import category_encoders as ce
from sklearn import metrics

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier

from sklearn.tree import export_graphviz
from six import StringIO  
from IPython.display import Image  
import pydotplus

from library.sb_utils import save_file
import json

In [2]:
# import the original full df, drop  useless/redundant columns
df = pd.read_csv('../data/processed/full_data_cleaned.csv')
df = df.drop(columns = ['product_id', 'aisle_id', 'department_id', 'eval_set'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33819106 entries, 0 to 33819105
Data columns (total 11 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   order_by_user_sequence  int64  
 3   order_dow               int64  
 4   order_hour_of_day       int64  
 5   days_since_prior_order  float64
 6   add_to_cart_sequence    int64  
 7   reordered               int64  
 8   product_name            object 
 9   aisle_name              object 
 10  dept_name               object 
dtypes: float64(1), int64(7), object(3)
memory usage: 2.8+ GB


In [3]:
# Get dictionaries connecting product-aisle-dept

with open('../data/processed/dicts/aisle_dept_dict.txt', 
          'r') as ad_file:
     ad_dict = json.load(ad_file)

with open('../data/processed/dicts/prod_aisle_dict.txt', 
          'r') as pa_file:
     pa_dict = json.load(pa_file)
        
with open('../data/processed/dicts/dept_id_name_dict.txt', 
          'r') as dd_file:
     dd_dict = json.load(dd_file)
        
with open('../data/processed/dicts/aisle_id_name_dict.txt', 
          'r') as aa_file:
     aa_dict = json.load(aa_file)
        
with open('../data/processed/dicts/prod_id_name_dict.txt', 
          'r') as pp_file:
     pp_dict = json.load(pp_file)
        
dd_dict

{'7': 'beverages',
 '16': 'dairy eggs',
 '19': 'snacks',
 '17': 'household',
 '4': 'produce',
 '14': 'breakfast',
 '13': 'pantry',
 '20': 'deli',
 '1': 'frozen',
 '11': 'personal care',
 '12': 'meat seafood',
 '6': 'international',
 '3': 'bakery',
 '15': 'canned goods',
 '9': 'dry goods pasta',
 '5': 'alcohol',
 '8': 'pets',
 '18': 'babies',
 '2': 'other',
 '21': 'missing',
 '10': 'bulk'}

In [4]:
# Fix dictionary to make the keys int rather than str

pp_dict = {int(k):v for k,v in pp_dict.items()}
aa_dict = {int(k):v for k,v in aa_dict.items()}
dd_dict = {int(k):v for k,v in dd_dict.items()}
pa_dict = {int(k):v for k,v in pa_dict.items()}
ad_dict = {int(k):v for k,v in ad_dict.items()}

dd_dict

{7: 'beverages',
 16: 'dairy eggs',
 19: 'snacks',
 17: 'household',
 4: 'produce',
 14: 'breakfast',
 13: 'pantry',
 20: 'deli',
 1: 'frozen',
 11: 'personal care',
 12: 'meat seafood',
 6: 'international',
 3: 'bakery',
 15: 'canned goods',
 9: 'dry goods pasta',
 5: 'alcohol',
 8: 'pets',
 18: 'babies',
 2: 'other',
 21: 'missing',
 10: 'bulk'}

Decide what chunk of data to work with for the remainder of the project. Randomly choose users of some quantity to leave me with a df sized to function with the computer. Don't start out separating it into train/test split. My intuition is that cross-row calculations don't count as leakage and negatively impact modeling if I'm adding data that has to do with past orders. If this logic turns out to be inappropriate, I can just come back and split the set into separate users (or into certain orders per user) and re-run any subsequent code. 

In [5]:
# How many total users are there?
len(df['user_id'].unique())

206209

In [6]:
# 1% (appx. 2k) users seems like a reasonable place to start. Just see what happens after I add rows. 
# Randomly select 1% of users.
users = random.sample(list(set(df['user_id'].unique())), 2062)
df = df.loc[df['user_id'].isin(users), :]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 351399 entries, 52947 to 33799613
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   order_id                351399 non-null  int64  
 1   user_id                 351399 non-null  int64  
 2   order_by_user_sequence  351399 non-null  int64  
 3   order_dow               351399 non-null  int64  
 4   order_hour_of_day       351399 non-null  int64  
 5   days_since_prior_order  330558 non-null  float64
 6   add_to_cart_sequence    351399 non-null  int64  
 7   reordered               351399 non-null  int64  
 8   product_name            351399 non-null  object 
 9   aisle_name              351399 non-null  object 
 10  dept_name               351399 non-null  object 
dtypes: float64(1), int64(7), object(3)
memory usage: 32.2+ MB


In [8]:
# Before adding rows, I'll need a dictionary of details to include in each new row.

df.groupby('user_id', 'order_by_user_sequence')[[order_id, user_id, order_by_user_sequence, order_dow]]

ValueError: Index data must be 1-dimensional

In [ ]:
# Add rows so that every order contains ever product ever ordered, with new rows as non-orders.
# In the processing of editing this to make it work for df here with more users.
for user in users:
    # Make sure to only work with one user's orders at a time.
    rows_to_work_with = df[df['user_id']==user]
    for n in range(2,100):
        # Get items from order n not reordered in order n+1
        order_n = rows_to_work_with[rows_to_work_with['order_by_user_sequence']==n]['product_id'].unique().tolist()
        order_n1 = rows_to_work_with[rows_to_work_with['order_by_user_sequence']==(n+1)]['product_id'].unique().tolist()
        only_n = [x for x in order_n if x not in order_n1]
        # Get n1 deets from the big deets dict
        order_n1_deets = orders_deets.get(n+1)
        # Add to n1 deets dict with product ids from order_n
        order_n1_deets.update({'product_id': only_n})
        # Turn dict into df of new rows
        order_n1_new_rows = pd.DataFrame.from_dict(order_n1_deets)
        # Add new rows to df
        df = pd.concat([df, order_n1_new_rows])


In [ ]:
# Create reorders_so_far column.
# Code I used with the practice user:

previous_round_items = set(df[df['reorders_so_far']==3][
    'product_name'])

grouped_by_product = df[df['reorders_so_far']==3].groupby(
    'product_name')['order_by_user_sequence']
df['keep'] = df.assign(min=grouped_by_product.transform(min))['min']

for n in range (4,47):
    for order in range(7,101):
        items_this_order = set(products_reordered_each_order[order])
        reordered_this_iteration = set(
            previous_round_items.intersection(items_this_order))
        rows_to_change = df.loc[(df.loc[
        :,'order_by_user_sequence']==order) & (df.loc[
        :, 'reorders_so_far']==n-1) & (df['order_by_user_sequence']!=
        df['keep']) & (df.loc[:,'product_name'].isin(
        reordered_this_iteration))]
        df.loc[rows_to_change.index, 'reorders_so_far'] = n
    previous_round_items = set(df[df['reorders_so_far']==n][
        'product_name'])
    grouped_by_product = df[df['reorders_so_far']==n].groupby(
        'product_name')['order_by_user_sequence']
    df['keep'] = df.assign(min=grouped_by_product.transform(min))[
        'min']

In [ ]:
# Create past_orders column & delete reorders_so_far
# Code I used to create past_orders with one user:

items_already_ordered_ntimes = set(df[df['past_orders']==2][
    'product_name'])
when_items_first_ordered = list(df.groupby('product_name')[
    'order_by_user_sequence'].idxmin())

for n in range(2,47): 
    rows_npast = df[df['product_name'].isin(
    items_already_ordered_ntimes)]
    rows_npast = rows_npast[rows_npast['past_orders']==0]
    rows_npast = rows_npast.drop(when_items_first_ordered, axis=0, 
                             errors='ignore')
    when_reordered_ntimes = df[df['reorders_so_far']==n].set_index(
    'product_name').to_dict()['order_by_user_sequence']
    for prod, order in when_reordered_ntimes.items():
        ind_to_delete = rows_npast[(rows_npast['product_name']==prod) &
                                   (rows_npast[
                                       'order_by_user_sequence']
                                    >order)].index
        rows_npast = rows_npast.drop(ind_to_delete)
    df.loc[np.array(rows_npast.index),'past_orders'] = n
    items_already_ordered_ntimes = set(df[df['past_orders']==n][
        'product_name'])

In [ ]:
# Engineer columns for product keywords

In [ ]:
# Change format of dow, hour columns

In [ ]:
# Save work done so far as new csv file

In [ ]:
# Move to new notebook for encoding & standardizing remaining features